In [ ]:
# Ячейка 1: Импорт необходимых библиотек
import pandas as pd
import numpy as np
import os
import glob

# Импорт моделей
from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier

# Импорт метрик для оценки
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Импорт для разделения выборки
from sklearn.model_selection import train_test_split, TimeSeriesSplit # <--- Добавлено TimeSeriesSplit

# Настройки для отображения Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# Загрузка данных

# Путь к папке с файлами
data_path = 'data/features_final/'

# Загрузка индивидуальных файлов (используем SBER и ROSN как пример)
try:
    sber_df = pd.read_csv(os.path.join(data_path, 'SBER_final.csv'))
    rosn_df = pd.read_csv(os.path.join(data_path, 'ROSN_final.csv'))
    print(f"SBER_final.csv загружен: {sber_df.shape}")
    print(f"ROSN_final.csv загружен: {rosn_df.shape}")
except FileNotFoundError as e:
    print(f"Ошибка при загрузке индивидуальных файлов: {e}. Убедитесь, что файлы существуют.")
    # Создаем пустые датафреймы, если файлы не найдены, чтобы код ниже не падал
    sber_df = pd.DataFrame()
    rosn_df = pd.DataFrame()

# Объединение всех файлов из папки
all_files = glob.glob(os.path.join(data_path, "*_final.csv"))

if not all_files:
    print(f"В папке {data_path} не найдено файлов *_final.csv для объединения.")
    all_stocks_df = pd.DataFrame() # Создаем пустой датафрейм
else:
    df_list = []
    for filename in all_files:
        try:
            df_temp = pd.read_csv(filename, index_col=None, header=0)
            df_list.append(df_temp)
        except Exception as e:
            print(f"Ошибка при чтении файла {filename}: {e}")
    
    if df_list:
        all_stocks_df = pd.concat(df_list, axis=0, ignore_index=True)
        print(f"\nВсего {len(df_list)} файлов объединено в all_stocks_df.")
        print(f"Размер объединенного датасета: {all_stocks_df.shape}")
        # print("\nПервые 5 строк объединенного датасета:")
        # print(all_stocks_df.head())
        # print("\nИнформация о типах данных и пропусках в объединенном датасете:")
        # all_stocks_df.info()
    else:
        print("Не удалось загрузить ни один файл для объединения.")
        all_stocks_df = pd.DataFrame()

# Вывод информации для примера (можно раскомментировать для проверки)
# if not sber_df.empty:
#     print("\nИнформация по SBER_final.csv:")
#     sber_df.info()
# if not rosn_df.empty:
#     print("\nИнформация по ROSN_final.csv:")
#     rosn_df.info()

In [ ]:
# Определение списков признаков и целевых переменных

# На основе data/final_structure.txt

# Колонки, которые не являются ни признаками, ни стандартными целями
on_feature_cols = ['date', 'SECID'] 

# --- Компоненты признаков ---
price_volume_cols = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME', 'WAPRICE']
ma_cols = ['SMA_5', 'EMA_5', 'SMA_10', 'EMA_10', 'SMA_20', 'EMA_20', 'SMA_50', 'EMA_50', 'SMA_200', 'EMA_200']
oscillator_cols = [
    'RSI', 'MACD', 'MACD_Signal', 'MACD_Hist', 'BB_Middle', 'BB_Upper', 'BB_Lower', 'BB_Width',
    'STOCH_K', 'STOCH_D', 'ATR', 'VWAP', 'OBV', 'OBV_MA', 'Williams_%R', 'Momentum',
    'Plus_DI', 'Minus_DI', 'ADX', 'MFI', 'PVO', 'PVO_Signal', 'PVO_Hist', 'Chaikin_AD',
    'Chaikin_Oscillator', 'CCI', 'EMV', 'A/D_Line', 'Bull_Power', 'Bear_Power', 'TEMA'
]
fundamental_cols = [
    'Assets_q', 'Assets_y', 'CAPEX_q', 'CAPEX_y', 'Cash_q', 'Cash_y', 'Debt_q', 'Debt_y',
    'DividendsPaid_q', 'DividendsPaid_y', 'EBITDA_q', 'EBITDA_y', 'Equity_q', 'Equity_y',
    'NetDebt_q', 'NetDebt_y', 'NetProfit_q', 'NetProfit_y', 'OperatingCashFlow_q', 'OperatingCashFlow_y',
    'OperatingExpenses_q', 'OperatingExpenses_y', 'OperatingProfit_q', 'OperatingProfit_y',
    'Revenue_q', 'Revenue_y'
]
macro_index_cols = [
    'BRENT_CLOSE', 'KEY_RATE', 'USD_RUB', 'MRBC', 'RTSI', 'MCXSM', 'IMOEX', 'MOEXBC',
    'MOEXBMI', 'MOEXCN', 'MOEXIT', 'MOEXRE', 'MOEXEU', 'MOEXFN', 'MOEXINN', 'MOEXMM',
    'MOEXOG', 'MOEXTL', 'MOEXTN', 'MOEXCH'
]
relative_coeff_cols = [
    'ROE_y', 'ROA_y', 'EBITDA_Margin_y', 'NetProfit_Margin_y', 'Debt_Equity_q', 'Debt_Equity_y',
    'NetDebt_EBITDA_y_q', 'NetDebt_EBITDA_y_y', 'EPS_y', 'BVPS_q', 'BVPS_y', 'SPS_y',
    'PE_y', 'PB_q', 'PB_y', 'PS_y', 'EV_EBITDA_y'
]
X_media_features = [
    'score_blog', 'score_blog_roll_avg_15', 'score_blog_roll_avg_50',
    'Index_MOEX_blog_score', 'Avg_Other_Indices_blog_score',
    'Avg_Other_Indices_blog_score_roll_avg_15', 'Avg_Other_Indices_blog_score_roll_avg_50', 
    'score_news', 'score_news_roll_avg_15', 'score_news_roll_avg_50',
    'Index_MOEX_news_score', 'Avg_Other_Indices_news_score', 
    'Avg_Other_Indices_news_score_roll_avg_15', 'Avg_Other_Indices_news_score_roll_avg_50'
]

# --- Основные наборы признаков ---
# 1. Базовый набор (все, кроме медиа)
X_base_features = price_volume_cols + ma_cols + oscillator_cols + fundamental_cols + macro_index_cols + relative_coeff_cols

# 2. Расширенный набор (базовый + медиа)
X_extended_features = X_base_features + X_media_features

# --- Новые наборы признаков для дополнительных сценариев ---
# 3. Технические индикаторы + Макро (Базовый - Фундаментальные - Относительные коэффициенты)
# (media_features не входят в X_base_features, поэтому их отдельно убирать не надо из X_base_features)
X_TECH_MACRO_features = list(set(price_volume_cols + ma_cols + oscillator_cols + macro_index_cols))

# 4. Только технические индикаторы (из цен, средних, осцилляторов)
# (X_TECH_MACRO_features - Макро)
X_TECH_ONLY_features = list(set(price_volume_cols + ma_cols + oscillator_cols))


# --- Целевые переменные ---
y_reg_targets = ['target_1d', 'target_3d', 'target_7d', 'target_30d', 'target_180d']
y_clf_binary_targets = ['target_1d_binary', 'target_3d_binary', 'target_7d_binary', 'target_30d_binary', 'target_180d_binary']

# Колонки, предназначенные для записи будущих предсказаний моделей
existing_prediction_cols = [
    'target_1d_pred', 'target_3d_pred', 'target_7d_pred', 'target_30d_pred', 'target_180d_pred',
    'target_1d_binary_pred', 'target_3d_binary_pred', 'target_7d_binary_pred',
    'target_30d_binary_pred', 'target_180d_binary_pred'
]

print(f"Количество базовых признаков (X_base): {len(X_base_features)}")
print(f"Количество медиа-признаков (X_media): {len(X_media_features)}")
print(f"Количество расширенных признаков (X_extended): {len(X_extended_features)}")
print(f"Количество признаков в наборе Тех.Индикаторы+Макро (X_TECH_MACRO): {len(X_TECH_MACRO_features)}")
print(f"Количество признаков в наборе Только Тех.Индикаторы (X_TECH_ONLY): {len(X_TECH_ONLY_features)}")

print(f"\nЦелевые для регрессии: {y_reg_targets}")
print(f"Целевые для классификации: {y_clf_binary_targets}")
print(f"Колонки для будущих предсказаний (не проверяются на наличие в исходных CSV): {existing_prediction_cols}")

# Колонки, которые ДОЛЖНЫ БЫТЬ в CSV для корректной работы скрипта:
required_cols_for_script = list(set(
    non_feature_cols + 
    price_volume_cols + ma_cols + oscillator_cols + fundamental_cols + macro_index_cols + relative_coeff_cols + # компоненты X_base
    X_media_features + # компоненты X_extended
    y_reg_targets + y_clf_binary_targets
))

if not all_stocks_df.empty:
    missing_required_cols_in_loaded_df = []
    for col in required_cols_for_script:
        if col not in all_stocks_df.columns:
            missing_required_cols_in_loaded_df.append(col)
            
    if missing_required_cols_in_loaded_df:
        print(f"\nВНИМАНИЕ: Следующие НЕОБХОДИМЫЕ для работы скрипта колонки НЕ найдены в загруженном all_stocks_df: {set(missing_required_cols_in_loaded_df)}")
        print("Это может означать, что CSV файлы не содержат всех нужных признаков или целевых переменных, либо есть опечатки в именах.")
        print("Пожалуйста, проверьте ваши CSV файлы и определения колонок в Ячейке 3.")
        if 'NetDebt_EBITDA_y_y' in missing_required_cols_in_loaded_df:
            print("-> Среди отсутствующих есть 'NetDebt_EBITDA_y_y'. Это важный базовый признак. Проверьте его наличие и название в CSV.")
    else:
        print("\nВсе НЕОБХОДИМЫЕ для обучения колонки (идентификаторы, признаки, актуальные таргеты) присутствуют в all_stocks_df.")
else:
    print("\nПроверка необходимых колонок не проводилась, так как all_stocks_df пуст (данные еще не загружены или не найдены).")

print("\nСписки признаков (включая новые сценарии X_TECH_MACRO, X_TECH_ONLY) и целевых переменных определены.")

In [ ]:
# Вспомогательная функция для метрики Sgn Acc

def sign_accuracy(y_true, y_pred):
    """
    Рассчитывает точность предсказания знака.
    1, если знаки совпадают (или оба 0), 0 иначе.
    """
    y_true_sign = np.sign(y_true)
    y_pred_sign = np.sign(y_pred)
    return np.mean(y_true_sign == y_pred_sign)

print("Функция sign_accuracy определена.")

In [ ]:
# Ячейка 5: Функция для обучения и оценки моделей

def train_evaluate_model(df, ticker_name, target_col, model_type, feature_set_name, current_feature_list, random_state=42):
    """
    Обучает и оценивает модель на ЗАДАННОМ списке признаков.
    Обрабатывает NaN в целевой переменной.
    Использует TimeSeriesSplit для разделения данных (или последние 15% для теста, если данных мало).
    Не заполняет NaN в признаках (LightGBM и XGBoost обработают их).

    Параметры:
    - df: DataFrame с данными.
    - ticker_name: Название тикера или 'ALL_STOCKS' для объединенного датасета.
    - target_col: Название целевой переменной.
    - model_type: 'LGBMRegressor', 'XGBRegressor', 'LGBMClassifier', 'XGBClassifier'.
    - feature_set_name: Имя набора признаков (для логирования/результатов, например, 'Base', 'Extended', 'TechOnly').
    - current_feature_list: СПИСОК названий колонок признаков для использования.
    - random_state: Для воспроизводимости моделей.

    Возвращает:
    - Словарь с метриками.
    """
    results = {
        'Ticker': ticker_name,
        'TargetVariable': target_col,
        'ModelType': model_type,
        'FeatureSet': feature_set_name, # Имя набора признаков для отчета
        'NumTrainSamples': 0,
        'NumTestSamples': 0,
        'MAE': np.nan, 'RMSE': np.nan, 'R2': np.nan, 'Sgn_acc': np.nan, # Для регрессии
        'Accuracy': np.nan, 'Precision': np.nan, 'Recall': np.nan, 'F1': np.nan, 'ROC_AUC': np.nan # Для классификации
    }

    # 1. Создание признаков X и цели y
    # Убедимся, что все признаки из current_feature_list существуют в df
    final_feature_list_for_df = [f for f in current_feature_list if f in df.columns]
    
    if len(final_feature_list_for_df) < len(current_feature_list):
        missing_from_list = set(current_feature_list) - set(final_feature_list_for_df)
        print(f"Предупреждение для {ticker_name}, таргет {target_col}, набор {feature_set_name}: \
              Некоторые признаки из заданного списка {feature_set_name} отсутствуют в DataFrame: {missing_from_list}.\
              Будут использованы только доступные: {len(final_feature_list_for_df)} из {len(current_feature_list)}.")

    if not final_feature_list_for_df: # Если после фильтрации не осталось признаков
        print(f"КРИТИЧЕСКОЕ ПРЕДУПРЕЖДЕНИЕ для {ticker_name}, таргет {target_col}, набор {feature_set_name}: \
              Ни один из признаков из списка {feature_set_name} не найден в DataFrame. Пропуск этого сценария.")
        results['MAE'] = 'NoFeaturesInDf' if model_type.endswith('Regressor') else np.nan
        results['Accuracy'] = 'NoFeaturesInDf' if model_type.endswith('Classifier') else np.nan
        return results

    X = df[final_feature_list_for_df].copy()
    y = df[target_col].copy()

    # 2. Удаление строк с NaN в целевой переменной
    valid_indices = y.dropna().index
    X = X.loc[valid_indices]
    y = y.loc[valid_indices]

    if X.empty or y.empty or len(X) < 2: 
        print(f"Предупреждение для {ticker_name}, таргет {target_col}, набор {feature_set_name}: После удаления NaN в таргете не осталось данных или их слишком мало ({len(X)}).")
        results['MAE'] = 'NoData' if model_type.endswith('Regressor') else np.nan
        results['Accuracy'] = 'NoData' if model_type.endswith('Classifier') else np.nan
        return results

    if X.shape[1] == 0: # Если после .loc[valid_indices] не осталось колонок (маловероятно, если final_feature_list_for_df не пуст)
        print(f"Предупреждение для {ticker_name}, таргет {target_col}, набор {feature_set_name}: Не осталось признаков после обработки NaN в таргете.")
        results['MAE'] = 'NoFeaturesPostTargetNaN' if model_type.endswith('Regressor') else np.nan
        results['Accuracy'] = 'NoFeaturesPostTargetNaN' if model_type.endswith('Classifier') else np.nan
        return results

    # 4. Разделение на обучающую и тестовую выборки
    n_splits_for_ts = 5 
    min_samples_for_tscv = n_splits_for_ts + 1

    if len(X) < min_samples_for_tscv:
        print(f"Предупреждение для {ticker_name}, таргет {target_col}, набор {feature_set_name}: Слишком мало данных ({len(X)}) для TimeSeriesSplit. Используем простой split (~15% тест)." )
        test_sample_size = int(len(X) * 0.15)
        if test_sample_size == 0 and len(X) > 1: test_sample_size = 1
        if len(X) - test_sample_size < 1 or test_sample_size < 1:
            print(f"Предупреждение для {ticker_name}, таргет {target_col}, набор {feature_set_name}: Недостаточно данных для простого разделения.")
            results['MAE'] = 'NoTrainTestData' if model_type.endswith('Regressor') else np.nan
            results['Accuracy'] = 'NoTrainTestData' if model_type.endswith('Classifier') else np.nan
            return results
        train_indices = X.index[:len(X)-test_sample_size]
        test_indices = X.index[len(X)-test_sample_size:]
    else:
        tscv = TimeSeriesSplit(n_splits=n_splits_for_ts)
        all_splits = list(tscv.split(X, y))
        train_indices, test_indices = all_splits[-1] 
        
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]

    results['NumTrainSamples'] = len(X_train)
    results['NumTestSamples'] = len(X_test)

    if len(X_train) == 0 or len(X_test) == 0:
        print(f"Предупреждение для {ticker_name}, таргет {target_col}, набор {feature_set_name}: Пустой train или test набор.")
        results['MAE'] = 'NoTrainTestData' if model_type.endswith('Regressor') else np.nan
        results['Accuracy'] = 'NoTrainTestData' if model_type.endswith('Classifier') else np.nan
        return results

    # 5. Инициализация и обучение модели
    model = None
    is_regressor = False
    if model_type == 'LGBMRegressor':
        model = LGBMRegressor(random_state=random_state, verbosity=-1)
        is_regressor = True
    elif model_type == 'XGBRegressor':
        model = XGBRegressor(random_state=random_state)
        is_regressor = True
    elif model_type == 'LGBMClassifier':
        model = LGBMClassifier(random_state=random_state, verbosity=-1)
    elif model_type == 'XGBClassifier':
        model = XGBClassifier(random_state=random_state, eval_metric='logloss' if pd.unique(y_train).size <= 2 else None)
    else:
        raise ValueError(f"Неизвестный тип модели: {model_type}")

    try:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        if is_regressor:
            results['MAE'] = mean_absolute_error(y_test, y_pred)
            results['RMSE'] = np.sqrt(mean_squared_error(y_test, y_pred))
            results['R2'] = r2_score(y_test, y_pred)
            results['Sgn_acc'] = sign_accuracy(y_test, y_pred)
        else: 
            results['Accuracy'] = accuracy_score(y_test, y_pred)
            results['Precision'] = precision_score(y_test, y_pred, zero_division=0)
            results['Recall'] = recall_score(y_test, y_pred, zero_division=0)
            results['F1'] = f1_score(y_test, y_pred, zero_division=0)
            if hasattr(model, "predict_proba"):
                if len(np.unique(y_test)) <= 2:
                    y_pred_proba = model.predict_proba(X_test)[:, 1]
                    results['ROC_AUC'] = roc_auc_score(y_test, y_pred_proba)
                else: results['ROC_AUC'] = np.nan
            else: results['ROC_AUC'] = np.nan
                
    except Exception as e:
        print(f"Ошибка при обучении/оценке модели {model_type} для {ticker_name}, таргет {target_col}, набор {feature_set_name}: {e}")
        error_msg_short = str(e).splitlines()[0][:70] 
        if is_regressor: results['MAE'] = f'Error: {error_msg_short}'
        else: results['Accuracy'] = f'Error: {error_msg_short}'

    return results

print("Функция train_evaluate_model обновлена для приема списка признаков current_feature_list.")

In [ ]:
# Ячейка 6: Запуск цикла исследования

all_results_list = []

dataframes_to_process = {}
if not sber_df.empty: dataframes_to_process['SBER'] = sber_df
if not rosn_df.empty: dataframes_to_process['ROSN'] = rosn_df
if not all_stocks_df.empty: dataframes_to_process['ALL_STOCKS'] = all_stocks_df
else: print("Нет данных для обработки в sber_df, rosn_df или all_stocks_df.")

model_types_reg = ['LGBMRegressor', 'XGBRegressor']
model_types_clf = ['LGBMClassifier', 'XGBClassifier']

# Определяем словарь с наборами признаков для итерации
# Ключ - имя набора (для отчета), Значение - список признаков
feature_set_scenarios = {
    "Base": X_base_features,
    "Extended": X_extended_features,
    "Tech_Macro": X_TECH_MACRO_features,
    "Tech_Only": X_TECH_ONLY_features
}

if not dataframes_to_process:
    print("Нет датафреймов для обработки. Проверьте загрузку данных.")
else:
    print(f"Начинаем исследование для: {list(dataframes_to_process.keys())}")
    print(f"Будут протестированы следующие наборы признаков: {list(feature_set_scenarios.keys())}\n")

    for df_name, current_df in dataframes_to_process.items():
        print(f"--- Обработка датафрейма: {df_name} ---")
        
        for target in y_reg_targets:
            if target not in current_df.columns:
                print(f"Целевая переменная {target} отсутствует в {df_name}. Пропуск.")
                continue
            for model_name in model_types_reg:
                for f_set_name, f_set_list in feature_set_scenarios.items(): # Итерация по словарю наборов признаков
                    print(f"Обучение (Регрессия): {model_name} на {f_set_name} ({len(f_set_list)} признаков) для {target} ({df_name})")
                    results = train_evaluate_model(current_df, df_name, target, model_name, 
                                                   f_set_name, # Имя набора для отчета
                                                   f_set_list  # Список признаков для этого набора
                                                  )
                    all_results_list.append(results)
                    # Выводим одну из ключевых метрик для быстрой проверки
                    print(f"  Результат: MAE={results.get('MAE', 'N/A')}, RMSE={results.get('RMSE', 'N/A')}, R2={results.get('R2', 'N/A')}, Sgn_acc={results.get('Sgn_acc', 'N/A')}")

        for target in y_clf_binary_targets:
            if target not in current_df.columns:
                print(f"Целевая переменная {target} отсутствует в {df_name}. Пропуск.")
                continue
            for model_name in model_types_clf:
                for f_set_name, f_set_list in feature_set_scenarios.items(): # Итерация по словарю наборов признаков
                    print(f"Обучение (Классификация): {model_name} на {f_set_name} ({len(f_set_list)} признаков) для {target} ({df_name})")
                    results = train_evaluate_model(current_df, df_name, target, model_name, 
                                                   f_set_name, # Имя набора для отчета
                                                   f_set_list  # Список признаков для этого набора
                                                  )
                    all_results_list.append(results)
                    # Выводим одну из ключевых метрик для быстрой проверки
                    print(f"  Результат: Acc={results.get('Accuracy', 'N/A')}, F1={results.get('F1', 'N/A')}, ROC_AUC={results.get('ROC_AUC', 'N/A')}")
        print(f"--- Обработка датафрейма {df_name} завершена ---\n")

    results_df = pd.DataFrame(all_results_list)
    print("\n--- Исследование завершено. Итоговая таблица результатов: ---")
    # Убедимся, что все нужные колонки есть перед выводом, или выведем все, что есть
    cols_to_show = ['Ticker', 'TargetVariable', 'ModelType', 'FeatureSet', 'NumTrainSamples', 'NumTestSamples', 
                    'MAE', 'RMSE', 'R2', 'Sgn_acc', 
                    'Accuracy', 'Precision', 'Recall', 'F1', 'ROC_AUC']
    existing_cols_in_results = [col for col in cols_to_show if col in results_df.columns]
    print(results_df[existing_cols_in_results].head())
    
    results_filename = "market_genius_model_results_v2.csv" # Новое имя файла для новых результатов
    results_df.to_csv(results_filename, index=False)
    print(f"\nРезультаты сохранены в файл: {results_filename}")

    if not results_df.empty:
        print("\nДетали по последним нескольким записям:")
        print(results_df[existing_cols_in_results].tail())

print("\nСкрипт завершил выполнение.")